<a href="https://colab.research.google.com/github/kumar2612/ai_ml/blob/ai_ml_basics_nov2024/langchain_tutorials/3_Classify_Text_into_Labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 7.6 MB/s eta 0:00:00


In [2]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 4.9 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"]="langchain_tutorials"

In [16]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

from langchain_groq import ChatGroq

#model = ChatGroq(model="llama3-8b-8192") # not working
model = ChatGroq(model="llama-3.3-70b-versatile") # working
#model = ChatGroq(model="llama3-70b-8192") #working

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# LLM
llm = model.with_structured_output(
    Classification
)

In [18]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

In [21]:
#with dictionary output
#inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
inp = """
गोवा की यात्रा बहुत अच्छी रही।
समुद्र तट बहुत गर्म थे।
मुझे समुद्र तट पर खेलने में बहुत मजा आया।
मेरी बेटी बहुत गुस्से में थी।
"""
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

#response.dict() # deperecated warning
response.model_dump()

{'sentiment': 'positive', 'aggressiveness': 2, 'language': 'Hindi'}

In [27]:
#classification model with finer control

class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text", enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        description="The language the text is written in", enum=["spanish", "english", "french", "german", "italian","hindi"]
    )

In [28]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

# LLM
llm = model.with_structured_output(
    Classification
)

In [29]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='happy', aggressiveness=1, language='spanish')

In [30]:
#with dictionary output
#inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
inp = """
गोवा की यात्रा बहुत अच्छी रही।
समुद्र तट बहुत गर्म थे।
मुझे समुद्र तट पर खेलने में बहुत मजा आया।
मेरी बेटी बहुत गुस्से में थी।
"""
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

#response.dict() # deperecated warning
response.model_dump()

{'sentiment': 'happy', 'aggressiveness': 1, 'language': 'hindi'}

In [31]:
inp = "Weather is ok here, I can go outside without much more than a coat"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='happy', aggressiveness=1, language='english')